# Laboratorium 6 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab6`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas matplotlib`

## Część 1. - przygotowanie danych

In [30]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas
import functools

from random import choice, sample
from statistics import mean, stdev

from reco_utils import *

In [2]:
# wczytujemy oceny uytkownikow i obliczamy (za pomocą collaborative filtering) wszystkie przewidywane oceny filmow

raw_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
movies = list(raw_ratings['movieId'].unique())
users = list(raw_ratings['userId'].unique())
ratings = get_predicted_ratings(raw_ratings)
ratings

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,10,9,9,7,7,9,10,6,6,9,...,6,8,7,8,8,6,9,10,6,8
2,6,5,5,4,4,5,6,3,4,5,...,3,5,4,5,4,4,5,5,3,4
3,4,4,4,3,3,4,4,2,3,4,...,2,4,3,4,3,3,4,4,3,4
4,8,7,8,6,6,7,8,5,5,7,...,5,7,5,7,6,5,7,8,5,6
5,6,6,6,4,5,5,6,4,4,5,...,3,5,4,5,5,4,6,6,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,9,8,8,6,7,8,9,5,6,8,...,5,7,6,7,6,5,8,9,6,7
607,9,8,8,6,7,7,8,5,5,7,...,5,7,6,7,6,5,8,8,6,7
608,8,7,7,5,6,7,7,5,5,6,...,4,6,5,6,5,5,7,7,5,6
609,5,5,5,4,4,5,5,3,3,5,...,3,4,4,4,4,4,5,5,4,4


In [3]:
# definiujemy testowe grupy uzytkownikow, dla ktorych bedziemy generowac rekomendacje

groups_no = 50
group_size = 5
groups = [sample(users, group_size) for i in range(groups_no)]
groups

[[79, 483, 130, 225, 52],
 [158, 546, 391, 239, 36],
 [454, 159, 127, 13, 228],
 [229, 167, 44, 591, 255],
 [179, 245, 226, 320, 200],
 [234, 338, 272, 60, 245],
 [550, 256, 347, 81, 440],
 [118, 64, 382, 12, 440],
 [424, 208, 99, 278, 181],
 [500, 171, 14, 397, 405],
 [483, 19, 341, 240, 562],
 [3, 478, 485, 31, 489],
 [382, 577, 44, 343, 483],
 [125, 593, 149, 581, 438],
 [340, 574, 476, 118, 201],
 [348, 559, 455, 468, 294],
 [155, 597, 36, 367, 137],
 [117, 25, 132, 312, 528],
 [325, 124, 295, 579, 145],
 [345, 42, 431, 126, 605],
 [147, 545, 363, 560, 168],
 [500, 542, 265, 51, 536],
 [432, 233, 170, 185, 216],
 [404, 359, 606, 447, 149],
 [566, 309, 117, 315, 478],
 [578, 118, 327, 586, 273],
 [40, 567, 589, 327, 374],
 [528, 45, 140, 55, 281],
 [469, 477, 147, 283, 192],
 [445, 529, 497, 413, 207],
 [77, 257, 266, 589, 57],
 [345, 332, 373, 193, 281],
 [453, 158, 364, 583, 456],
 [302, 445, 146, 289, 173],
 [199, 564, 554, 490, 358],
 [106, 174, 408, 475, 391],
 [585, 149, 380, 

## Część 2. - algorytmy proste

In [4]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania
    
class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'
        
    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [5]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'
    
    def recommend(self, movies, ratings, group, size):
        recommendations = movies
        recommendations.sort(key=lambda m: np.sum([ratings[m][user] for user in group]))
        return recommendations[:size]

In [6]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold
        
    def recommend(self, movies, ratings, group, size):
        recommendations = [movie for movie in movies if min([ratings[movie][user] for user in group]) > self.score_threshold]
        recommendations.sort(key=lambda m: np.average([ratings[m][user] for user in group]))
        return recommendations[:size]

In [7]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika

class DictatorshipRecommender(Recommender):
    def __init__(self, dictator_id):
        self.name = 'dictatorship'
        self.dictator_id = dictator_id
        
    def recommend(self, movies, ratings, group, size):
        recommendations = movies
        recommendations.sort(key=lambda m: ratings[m][self.dictator_id])
        return recommendations[:size]

In [8]:
# algorytm, ktory w kazdej turze uwzglednia preferencje tylko jednego, kolejnego uzytkownika

class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'
    
    def recommend(self, movies, ratings, group, size):
        recommendations = []
        for turn in range(size):
            user_id = group[turn%len(group)]
            recommendations.append(max(list(ratings.loc[user_id])))
        return recommendations

In [9]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class PAVRecommender(Recommender):
    def __init__(self, threshold):
        self.threshold = threshold
        self.name = 'PAV'
        
    def recommend(self, movies, ratings, group, size):
        satisfied_counter = np.ones(len(group))
        recommendations = []
        movies = [movie for movie in movies if min([ratings[movie][user] for user in group]) > self.threshold]
        for turn in range(size):
            scores = []
            for movie in movies:
                score = np.sum([1/satisfied_counter[idx] if ratings[movie][user]>self.threshold else 0 for idx, user in enumerate(group)])
                scores.append(score)
            best_movie_idx = max(range(len(scores)), key=scores.__getitem__)
            recommendations.append(movies[best_movie_idx])
            for idx, user in enumerate(group):
                satisfied_counter[idx] += 1 if ratings[recommendations[-1]][user] > self.threshold else 0
                
        return recommendations
            

## Część 3. - funkcje celu

In [99]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

@functools.cache
def top_n_movies_for_user(movies, user_id, n):
    recommendation = list(movies)
    recommendation.sort(key=lambda m: ratings[m][user_id])
    return recommendation[:n]


def total_score(recommendation, user_id, ratings):
    return np.sum([ratings[m][user_id] for m in recommendation])

In [100]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej

def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    return total_score(recommendation, user_id, ratings)/total_score(top_n_movies_for_user(tuple(movies), user_id, len(recommendation)), user_id, ratings)

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie

def overall_group_satisfaction(recommendation, group, movies, ratings):
    return 1.0 * sum([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]) / len(group)

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_dissatisfaction(recommendation, group, movies, ratings):
    satisfaction_scores = [overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]
    return max(satisfaction_scores) - min(satisfaction_scores)

## Część 4. - Sequential Hybrid Aggregation

In [113]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie
class SequentialHybridAggregationRecommender(Recommender):
    def __init__(self):
        self.name = 'sequential_hybrid_aggregation'
    
    def recommend(self, movies, ratings, group, size):
        alpha = 0.5
        recommendation = []
        avaliable_movies = movies
        for turn in range(size):
            print(f'Finding recommendation #{turn+1}')

            best_score = -1000000
            best_movie_idx = -1
            best_satisfaction = []
            for idx, movie in enumerate(avaliable_movies):
                if idx%10==0 or idx == len(avaliable_movies)-1:
                    percent = ((idx+1)/len(avaliable_movies))*100
                    bar = '{progress:-<50}'.format(progress='■'*math.floor(percent/2))
                    print(f'\rCalculating movie {movie:>7}: |{bar}| {round(percent,2)}%', end="")
                
                group_satisfaction = [overall_user_satisfaction(recommendation + [movie], user_id, movies, ratings) for user_id in group]
                score = (1-alpha)*np.average(group_satisfaction) + alpha*min(group_satisfaction)
                
                if score > best_score:
                    best_movie_idx = idx
                    best_score = score
                    best_satisfaction = group_satisfaction
            print("")
            recommendation.append(avaliable_movies[best_movie_idx])
            
            max(best_satisfaction)
            min(best_satisfaction)
            
            alpha = max(best_satisfaction) - min(best_satisfaction)
            avaliable_movies = [movie for movie in avaliable_movies if movie not in recommendation]
            
        return recommendation
            

## Część 5. - porównanie algorytmów

In [114]:
recommenders = [
    #RandomRecommender(),
    #AverageRecommender(),
    #AverageWithoutMiseryRecommender(2),
    #DictatorshipRecommender(1),
    #FairnessRecommender(),
    #PAVRecommender(2),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci funkcji celu dla kazdej rekomendacji
#  - obliczmy srednia i odchylenie standardowe wartosci funkcji celu
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    # ...
    print(f'Grading {recommender.name}')
    recommendations = [recommender.recommend(movies, ratings, group, recommendation_size) for group in groups]
    satisfactions = [overall_group_satisfaction(recommendation, group, movies, ratings) for recommendation, group in zip(recommendations, groups)]
    avg_satisfaction = np.average(satisfactions)
    std_satisfaction = np.std(satisfactions)
    print(f'Average satisfaciton: {avg_satisfaction} \n Standard deviation: {std_satisfaction}')

Grading sequential_hybrid_aggregation
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie   31150: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie   31150: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie   31150: |■■■■

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding r

Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie  117531: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

In [116]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(2),
    DictatorshipRecommender(1),
    FairnessRecommender(),
    PAVRecommender(2),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci funkcji celu dla kazdej rekomendacji
#  - obliczmy srednia i odchylenie standardowe wartosci funkcji celu
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    # ...
    print(f'Grading {recommender.name}')
    recommendations = [recommender.recommend(movies, ratings, group, recommendation_size) for group in groups]
    satisfactions = [overall_group_satisfaction(recommendation, group, movies, ratings) for recommendation, group in zip(recommendations, groups)]
    avg_satisfaction = np.average(satisfactions)
    std_satisfaction = np.std(satisfactions)
    print(f'Average satisfaciton: {avg_satisfaction} \n Standard deviation: {std_satisfaction}')

Grading random
Average satisfaciton: 1.8029328478724431 
 Standard deviation: 0.09104926180166176
Grading average
Average satisfaciton: 1.014586133393229 
 Standard deviation: 0.012337576300049384
Grading average_without_misery
Average satisfaciton: 1.1845430597440865 
 Standard deviation: 0.10230669529619171
Grading dictatorship
Average satisfaciton: 1.0341138309192544 
 Standard deviation: 0.016577014674225114
Grading fairness
Average satisfaciton: 1.802101413028621 
 Standard deviation: 0.13764985524651147
Grading PAV
Average satisfaciton: 1.164651817474527 
 Standard deviation: 0.1212467648078329
Grading sequential_hybrid_aggregation
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recomme

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie   58559: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding r

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie    6434: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie     593: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #6
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #7
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie   46772: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #8
Calculating movie     593: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #9
Calculating movie     593: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie     593: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #1
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #2
Calculating movie     318: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #3
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #4
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #5
Calculating movie    5644: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding 

Calculating movie    7560: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Finding recommendation #10
Calculating movie    7560: |■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 100.0%
Average satisfaciton: 2.5402187577291655 
 Standard deviation: 0.06773400672069366
